# Snowflake Account Usage - Workspace Version

### 이 앱은 여러분의 Snowflake 계정에 있는 account_usage 스키마를 기반으로 작동하도록 개발되었습니다. 

이 노트북은 Snowflake Notebooks에서 VS Code Workspace용으로 변환된 버전입니다.
SQL 셀 대신 Python에서 직접 SQL을 실행합니다.

자세한 정보는 아래 Snowflake 공식 문서 페이지를 참조하십시오.  
https://docs.snowflake.com/en/sql-reference/account-usage#account-usage-views

# 패키지 설정 및 측정 날짜 지정

In [ ]:
%pip install pandas plotly altair

In [ ]:
import pandas as pd
import plotly.express as px 
import datetime
import altair as alt
import plotly.graph_objects as go 
import numpy as np
from IPython.display import display, Markdown

from snowflake.snowpark.context import get_active_session

session = get_active_session()

def run_sql(sql):
    """Execute SQL and return pandas DataFrame"""
    return session.sql(sql).to_pandas()

def title(text):
    """Display title"""
    display(Markdown(f"## {text}"))

In [ ]:
title('모니터링 날짜 범위 지정')

DAYS = 30

today = datetime.date.today()
s = (today - datetime.timedelta(days=DAYS)).strftime('%Y-%m-%d')
e = today.strftime('%Y-%m-%d')

print(f"분석 기간: {s} ~ {e}")
print(f"\n※ 기간 변경하려면 위의 DAYS 값을 수정하세요 (7, 14, 30, 60, 120)")

# 사용량 개요

In [ ]:
title('Account 사용량 개요')

pandas_credits_used_df = run_sql(f"""
    SELECT round(sum(credits_used),0) as total_credits 
    FROM snowflake.account_usage.metering_history 
    WHERE start_time between '{s}' and '{e}'
""")

pandas_num_jobs_df = run_sql(f"""
    SELECT count(*) as number_of_jobs 
    FROM snowflake.account_usage.query_history 
    WHERE start_time between '{s}' and '{e}'
""")

pandas_current_storage_df = run_sql("""
    SELECT round(avg(storage_bytes + stage_bytes + failsafe_bytes) / power(1024, 4),2) as billable_tb 
    FROM snowflake.account_usage.storage_usage 
    WHERE USAGE_DATE = current_date() - 1
""")

credits_used_tile = pandas_credits_used_df.iloc[0].values[0]
num_jobs_tile = pandas_num_jobs_df.iloc[0].values[0]
current_storage_tile = pandas_current_storage_df.iloc[0].values[0]

print(f"📊 Credits Used: {int(credits_used_tile):,}")
print(f"📊 Total # of Jobs Executed: {int(num_jobs_tile):,}")
print(f"📊 Current Storage (TB): {current_storage_tile}")

In [ ]:
title('월별 전체 크레딧 사용량 추이')

credits_billed_df = run_sql("""
    SELECT date_trunc('MONTH', usage_date) as Usage_Month, 
           sum(CREDITS_BILLED) as sum_credits
    FROM snowflake.account_usage.metering_daily_history 
    WHERE usage_date >= DATEADD('MONTH', -12, DATE_TRUNC('MONTH', CURRENT_TIMESTAMP()))
    GROUP BY Usage_Month
""")

fig_credits_billed=px.bar(credits_billed_df,x='USAGE_MONTH',y='SUM_CREDITS', orientation='v',title="Credits Billed by Month")
fig_credits_billed.show()

print('The above chart is static and not modified by the date range filter')

# 가상 웨어하우스 (Virtual Warehouse) 분석

In [ ]:
title('지정된 기간 웨어하우스별 수행된 작업의 수')

pandas_jobs_by_warehouse_df = run_sql(f"""
    SELECT warehouse_name, count(*) as number_of_jobs 
    FROM snowflake.account_usage.query_history 
    WHERE start_time between '{s}' and '{e}' 
      AND warehouse_name is not null
    GROUP BY 1 
    ORDER BY 2 desc 
    LIMIT 10
""")

pandas_jobs_by_warehouse_df_sorted = pandas_jobs_by_warehouse_df.sort_values(by='NUMBER_OF_JOBS', ascending=True)

fig_jobs_by_warehouse=px.bar(pandas_jobs_by_warehouse_df_sorted,x='NUMBER_OF_JOBS',y='WAREHOUSE_NAME',orientation='h',title="# of Jobs by Warehouse")
fig_jobs_by_warehouse.update_traces(marker_color='purple')
fig_jobs_by_warehouse.show()

In [ ]:
title('지정된 기간 웨어하우스별 크레딧 사용양 추세')

pandas_credits_used_overtime_df = run_sql(f"""
    SELECT start_time::date as usage_date, warehouse_name, sum(credits_used) as total_credits_used 
    FROM snowflake.account_usage.warehouse_metering_history 
    WHERE start_time between '{s}' and '{e}' 
    GROUP BY 1,2 
    HAVING total_credits_used > 0.001
    ORDER BY 2,1
""")

fig_credits_used_overtime_df=px.bar(pandas_credits_used_overtime_df,x='USAGE_DATE',y='TOTAL_CREDITS_USED',color='WAREHOUSE_NAME',orientation='v',title="Credits Used Overtime")
fig_credits_used_overtime_df.show()

In [ ]:
title('지정된 기간 동안 웨어하우스별 워크로드 수행시간 분포')

ExeTimeInWH_df = run_sql(f"""
    WITH base_table as (
        SELECT warehouse_name, 
               total_elapsed_time,
               CASE
                    WHEN total_elapsed_time < 1 THEN 'a'
                    WHEN total_elapsed_time < 10 THEN 'b'
                    WHEN total_elapsed_time < 60 THEN 'c'
                    WHEN total_elapsed_time < 300 THEN 'd'
                    WHEN total_elapsed_time < 600 THEN 'e'
                    WHEN total_elapsed_time > 600 THEN 'f'
               END as exe_time
        FROM SNOWFLAKE.ACCOUNT_USAGE.QUERY_HISTORY
        WHERE warehouse_name is not null
          AND start_time between '{s}' and '{e}'
          AND warehouse_name not ilike 'compute_service%'   
    )
    SELECT warehouse_name, 
           sum(iff(exe_time= 'a', 1, 0)) U00_01S,
           sum(iff(exe_time= 'b', 1, 0)) U01_10S,
           sum(iff(exe_time= 'c', 1, 0)) U10_60S,
           sum(iff(exe_time= 'd', 1, 0)) U01_05M,
           sum(iff(exe_time= 'e', 1, 0)) U05_10M,
           sum(iff(exe_time= 'f', 1, 0)) U10M_
    FROM base_table
    GROUP BY warehouse_name
""")

plot_columns = ['U00_01S', 'U01_10S', 'U10_60S', 'U01_05M', 'U05_10M', 'U10M_']

labels_map = {
    'WAREHOUSE_NAME': 'Warehouse Name',
    'variable': 'Execution Time Bucket',
    'value': 'Number of Queries',
    'U00_01S': '< 1s',
    'U01_10S': '1s - 10s',
    'U10_60S': '10s - 60s',
    'U01_05M': '1m - 5m',
    'U05_10M': '5m - 10m',
    'U10M_': '> 10m'
}

df_long = pd.melt(ExeTimeInWH_df,
                  id_vars='WAREHOUSE_NAME',
                  value_vars=plot_columns,
                  var_name='variable',
                  value_name='value')

fig = px.bar(
    df_long,
    x='WAREHOUSE_NAME',
    y='value',
    color='variable',
    title='Query Execution Time Distribution by Warehouse',
    labels=labels_map,
    height=500,
    category_orders={'variable': plot_columns}
)

fig.show()

In [ ]:
title('일별 웨어하우스의 워크로드 추이 (지난 1개월)')

RunQueueWH_df = run_sql("""
    SELECT to_date(start_time) as date,
           warehouse_name,
           sum(avg_running) as sum_running,
           sum(avg_queued_load) as sum_queued
    FROM snowflake.account_usage.warehouse_load_history
    WHERE to_date(start_time) >= dateadd(month, -1, current_timestamp())
    GROUP BY 1,2
    ORDER BY 1,2
""")

fig_running = px.line(
    RunQueueWH_df, x='DATE', y='SUM_RUNNING', color='WAREHOUSE_NAME',
    title='Daily Sum of Average Running Queries by Warehouse',
    labels={'DATE': 'Date', 'SUM_RUNNING': 'Sum of Avg Running Queries', 'WAREHOUSE_NAME': 'Warehouse'},
    markers=True
)
fig_running.update_layout(hovermode='x unified')
fig_running.show()

fig_queued = px.line(
    RunQueueWH_df, x='DATE', y='SUM_QUEUED', color='WAREHOUSE_NAME',
    title='Daily Sum of Average Queued Queries by Warehouse',
    labels={'DATE': 'Date', 'SUM_QUEUED': 'Sum of Avg Queued Queries', 'WAREHOUSE_NAME': 'Warehouse'},
    markers=True
)
fig_queued.update_layout(hovermode='x unified')
fig_queued.show()

In [ ]:
title('지정된 기간동안 가장 오래 수행된 쿼리 리스트')

pandas_longest_queries_df = run_sql(f"""
    SELECT query_id, query_text, (execution_time / 1000) as exec_time 
    FROM snowflake.account_usage.query_history 
    WHERE execution_status = 'SUCCESS' 
      AND start_time between '{s}' and '{e}' 
      AND query_text != ''
      AND exec_time > 10
    ORDER BY execution_time desc 
    LIMIT 25
""")

max_len = 100
col_orig = 'QUERY_TEXT'
col_short = 'QUERY_TEXT_SHORT'

pandas_longest_queries_df[col_short] = pandas_longest_queries_df[col_orig].str.slice(0, max_len)
pandas_longest_queries_df.loc[pandas_longest_queries_df[col_orig].str.len() > max_len, col_short] += '...'

fig_longest_queries = px.bar(
    pandas_longest_queries_df, x='EXEC_TIME', y=col_short, orientation='h',
    title="Longest Successful Queries (Top 25)",
    hover_data={'EXEC_TIME': ':.2f s', col_orig: True, col_short: False},
    labels={col_short: "Query Text (Truncated)"}
)

fig_longest_queries.show()

In [ ]:
title('지정된 기간내의 반복되는 쿼리 수행')

Total_Execution_Time_df = run_sql(f"""
    SELECT query_text, 
           (sum(execution_time) / 1000) as exec_time,
           count(*) as number_of_query
    FROM snowflake.account_usage.query_history 
    WHERE execution_status = 'SUCCESS' 
      AND start_time between '{s}' and '{e}' 
      AND query_text != ''
    GROUP BY query_text 
    ORDER BY exec_time desc 
    LIMIT 10
""")

max_len = 100
col_orig = 'QUERY_TEXT'
col_short = 'QUERY_TEXT_SHORT'

Total_Execution_Time_df[col_short] = Total_Execution_Time_df[col_orig].str.slice(0, max_len)
Total_Execution_Time_df.loc[Total_Execution_Time_df[col_orig].str.len() > max_len, col_short] += '...'

fig_Total_Execution_Time = px.bar(
    Total_Execution_Time_df, x='EXEC_TIME', y=col_short, orientation='h',
    title="Total Execution Time by Repeated Queries",
    hover_data={'EXEC_TIME': ':.3f s', col_orig: True, col_short: False, 'NUMBER_OF_QUERY': ':,d'},
    labels={col_short: "Query Text (Truncated)", 'EXEC_TIME': "Total Execution Time (s)", 'NUMBER_OF_QUERY': "Number of Executions"}
)
fig_Total_Execution_Time.update_traces(marker_color='LightSkyBlue')

fig_Total_Execution_Time.show()

In [ ]:
title('지정된 기간동안 사용자별 평균 쿼리 실행 시간 (초)')

query_execution_df = run_sql(f"""
    SELECT user_name, (avg(execution_time)) / 1000 as average_execution_time 
    FROM snowflake.account_usage.query_history 
    WHERE execution_status = 'SUCCESS'
      AND start_time between '{s}' and '{e}'
    GROUP BY 1 
    ORDER BY 2 desc 
    LIMIT 10
""")

fig_cquery_execution=px.bar(query_execution_df,x='USER_NAME',y='AVERAGE_EXECUTION_TIME', orientation='v',title="Average Execution Time per User")
fig_cquery_execution.update_traces(marker_color='MediumPurple')
fig_cquery_execution.show()

# 클라우드 서비스 영역

In [ ]:
title('지정된 기간동안 웨어하우스별 클라우드서비스의 총 사용 크레딧양')

compute_gs_by_warehouse_df = run_sql(f"""
    SELECT warehouse_name, 
           sum(credits_used_cloud_services) CREDITS_USED_CLOUD_SERVICES 
    FROM snowflake.account_usage.warehouse_metering_history 
    WHERE start_time between '{s}' and '{e}'
    GROUP BY 1 
    ORDER BY 2 desc 
    LIMIT 10
""")

fig_compute_gs_by_warehouse=px.bar(compute_gs_by_warehouse_df,x='WAREHOUSE_NAME',y='CREDITS_USED_CLOUD_SERVICES', orientation='v',title="Compute and Cloud Services by Warehouse", barmode="group")
fig_compute_gs_by_warehouse.update_traces(marker_color='purple')
fig_compute_gs_by_warehouse.show()

In [ ]:
title('월별 스토리지 사용량 추이')

storage_overtime_df = run_sql("""
    SELECT date_trunc(month, usage_date) as usage_month, 
           avg(storage_bytes + stage_bytes + failsafe_bytes) / power(1024, 4) as billable_tb, 
           avg(storage_bytes) / power(1024, 4) as Storage_TB, 
           avg(stage_bytes) / power(1024, 4) as Stage_TB, 
           avg(failsafe_bytes) / power(1024, 4) as Failsafe_TB 
    FROM snowflake.account_usage.storage_usage 
    GROUP BY 1 
    ORDER BY 1
""")

storage_df_melted = pd.melt(storage_overtime_df, id_vars=['USAGE_MONTH'],
    value_vars=['STORAGE_TB', 'STAGE_TB', 'FAILSAFE_TB'], var_name='STORAGE_TYPE', value_name='SIZE_TB')

fig_stacked_storage = px.bar(storage_df_melted, x='USAGE_MONTH', y='SIZE_TB', color='STORAGE_TYPE',
    title="Data Storage used Overtime", orientation='v',
    labels={'USAGE_MONTH': 'Month', 'SIZE_TB': 'Storage Size (TB)', 'STORAGE_TYPE': 'Storage Type'})

fig_stacked_storage.show()
print('The above chart is static and not modified by the date range filter')

# 사용자

In [ ]:
title('지정된 기간동안 클라이언트 유형별, 사용자별 로그인 성공/실패 횟수')

logins_client_df_wide = run_sql(f"""
    SELECT reported_client_type as Client, 
           user_name, 
           sum(iff(is_success = 'NO', 1, 0)) as Failed, 
           count(*) as Success, 
           sum(iff(is_success = 'NO', 1, 0)) / nullif(count(*), 0) as login_failure_rate 
    FROM snowflake.account_usage.login_history 
    WHERE event_timestamp between '{s}' and '{e}'
    GROUP BY 1, 2 
    ORDER BY 5 desc
""")

logins_client_df_melted = pd.melt(logins_client_df_wide, id_vars=['CLIENT', 'USER_NAME'],
    value_vars=['SUCCESS', 'FAILED'], var_name='LOGIN_STATUS', value_name='COUNT')

fig_logins_client_grouped = px.bar(
    logins_client_df_melted, x='USER_NAME', y='COUNT', color='LOGIN_STATUS',
    facet_col='CLIENT', facet_col_wrap=3, title="Logins by Client",
    labels={'USER_NAME': 'Username', 'COUNT': 'Number of logins', 'LOGIN_STATUS': 'Login status', 'CLIENT': 'Client type'},
    color_discrete_map={'SUCCESS': 'mediumseagreen', 'FAILED': 'tomato'}, height=500)

fig_logins_client_grouped.show()

In [ ]:
title('지난 30일 이내 새로 로그인하지 않은 사용자')

session.sql("SHOW USERS").collect()
df_users_stale = run_sql("""
    SELECT * FROM TABLE(RESULT_SCAN(LAST_QUERY_ID()))
    WHERE DATEDIFF('Day',"last_success_login",CURRENT_DATE) > 30
""")

if not df_users_stale.empty:
    df_users_stale['last_success_login'] = pd.to_datetime(df_users_stale['last_success_login'])
    print("사용자 마지막 로그인 타임라인")
    
    fig_timeline = px.scatter(df_users_stale, x='last_success_login', y='name',
        title='Infrequent User',
        labels={'last_success_login': 'Last Success Login Date/Time', 'name': 'User Name'}, hover_name='name')
    fig_timeline.update_layout(height=max(400, len(df_users_stale['name'].unique()) * 20))
    fig_timeline.update_traces(marker=dict(size=8, symbol='circle'))
    fig_timeline.show()
else:
    print("조건에 맞는 사용자가 없습니다.")

In [ ]:
title('짧은 생명 주기의 테이블 리스트')

HighChurn_ShortLived_df = run_sql("""
    SELECT
            t.table_catalog||'.'||t.table_schema||'.'||t.table_name as fq_table_name,
            t.table_catalog as database,
            t.table_schema as schema,
            t.table_name as table_name,
            t.active_bytes/power(1024,3) as active_size_gb,
            t.time_travel_bytes/power(1024,3) as time_travel_gb,
            t.failsafe_bytes/power(1024,3) as failsafe_gb,
            t.retained_for_clone_bytes/power(1024,3) as clone_retain_gb,
            active_size_gb+time_travel_gb+failsafe_gb+clone_retain_gb as total_size_gb,
            (t.time_travel_bytes + t.failsafe_bytes + t.retained_for_clone_bytes)/power(1024,3) as non_active_size_gb,
            div0(non_active_size_gb,active_size_gb)*100 as churn_pct,
            t.deleted,
            timediff('hour',t.table_created,t.table_dropped) as table_life_duration_hours,
            t1.is_transient,
            t1.table_type,
            t1.retention_time,
            t1.last_altered
    FROM snowflake.account_usage.table_storage_metrics t
        JOIN snowflake.account_usage.tables t1
          ON t.id=t1.table_id
    WHERE 1=1
        AND t1.table_catalog not in ('SNOWFLAKE')
        AND (churn_pct>=40 OR table_life_duration_hours<=24)
    ORDER BY total_size_gb desc
""")

print("데이터베이스별 테이블 수")
fig_db = px.bar(HighChurn_ShortLived_df, x='DATABASE', color='DATABASE', title='데이터베이스별 테이블 수')
fig_db.show()

print("스키마별 테이블 수")
fig_schema = px.bar(HighChurn_ShortLived_df, x='SCHEMA', color='SCHEMA', title='스키마별 테이블 수')
fig_schema.show()

print("테이블 정보")
fig_table = go.Figure(data=[go.Table(
    header=dict(values=['DATABASE', 'SCHEMA', 'TABLE_NAME', 'DELETED', 'IS_TRANSIENT', 'LAST_ALTERED'], align='left'),
    cells=dict(values=[HighChurn_ShortLived_df['DATABASE'], HighChurn_ShortLived_df['SCHEMA'], 
               HighChurn_ShortLived_df['TABLE_NAME'], HighChurn_ShortLived_df['DELETED'],
               HighChurn_ShortLived_df['IS_TRANSIENT'], HighChurn_ShortLived_df['LAST_ALTERED']], align='left'))
])
fig_table.show()

In [ ]:
title('지난 7일간 쿼리에서 사용된 테이블 리스트')

TablesUsedInQuery_df = run_sql("""
    WITH access_history as (   
        SELECT distinct split(base.value:objectName, '.')[0]::string as DATABASE,
               split(base.value:objectName, '.')[1]::string as SCHEMA,
               split(base.value:objectName, '.')[2]::string as TABLE_NAME
        FROM snowflake.account_usage.access_history,
             lateral flatten (base_objects_accessed) base
        WHERE query_start_time between current_date()-7 and current_date()
    )
    SELECT tbl.table_catalog||'.'||tbl.table_schema||'.'||tbl.table_name as FQ_table_name,
           ah.database,
           ah.schema,
           ah.table_name,
           TO_NUMBER(tbl.bytes / POWER(1024,3),10,2) AS GB
    FROM snowflake.account_usage.tables tbl
    LEFT JOIN access_history ah
      ON tbl.table_name=ah.table_name
     AND tbl.table_schema=ah.schema
     AND tbl.table_catalog=ah.database
    WHERE ah.table_name is not NULL
      AND tbl.deleted is null
""")

df_sorted = TablesUsedInQuery_df.sort_values(by='GB', ascending=False)
fig_size = px.bar(df_sorted, y='DATABASE', x='GB', color='TABLE_NAME',
                 title='테이블별 총 크기 (GB)', orientation='h')
fig_size.show()

fig_table = go.Figure(data=[go.Table(
    header=dict(values=['DATABASE', 'SCHEMA', 'TABLE_NAME', 'GB'], align='left'),
    cells=dict(values=[df_sorted['DATABASE'], df_sorted['SCHEMA'], 
               df_sorted['TABLE_NAME'], df_sorted['GB']], align='left'))
])
fig_table.show()